# 整理省级行政区

In [5]:
import ijson
from geomet import wkt

file=open("./rdf/省份.ttl","a",encoding="utf-8")


with open('./矢量数据/省级行政区.geojson', 'r', encoding='utf-8') as f:
    objects = ijson.items(f, 'features.item')
    # 这个objects在这里就是相当于一个生成器，可以调用next函数取它的下一个值
    while True:
        try:
            # print(objects.__next__())

            data = objects.__next__()
            properties=data["properties"]
            geometry = data['geometry']
            towkt = wkt.dumps(geometry, decimals=8)
            # if data['properties']['省'] == "北京市":
            #     print(towkt)
            geometry=create_geometry(properties["省代码"],towkt)
            feature=create_feature(properties["省代码"],properties["省"],properties["类型"])
            file.write(geometry)
            file.write(feature)

        except StopIteration as e:
            print("数据读取完成")
            break

file.close()

数据读取完成


## 根据wkt创建geometry

\###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#geometry.polygon.region.110000
:geometry.polygon.region.110000 rdf:type owl:NamedIndividual ;
                                geosparql:asWKT """MULTIPOLYGON (((116.67527402 41.04010201, 116.67616420 41.04006172, 116.67679309 41.04012125, 116.67734482 41.04036821, 116.67786403 41.04065732, 116.67811217 41.04502816, 116.67091587 41.04108628, 116.67355366 41.04045864, 116.67500175 41.04014031, 116.67527402 41.04010201)))"""^^geosparql:wktLiteral .


In [1]:

def create_geometry(region_id,wkt_text):
    text=f'''
###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#geometry.polygon.region.{region_id}
:geometry.polygon.region.{region_id} rdf:type owl:NamedIndividual ;
                                geosparql:asWKT """{wkt_text}"""^^geosparql:wktLiteral .
'''

    return  text

## 根据geojson的属性值创建Feature


\###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#region.110000
:region.110000 rdf:type owl:NamedIndividual ,
                        :AdministrativeRegion_Province ;
               geosparql:hasGeometry :geometry.polygon.region.110000 ;
               :RegionID 110000 ;
               :RegionName "北京市" ;
               :RegionType "直辖市" ;
               rdfs:label "北京市" .

In [3]:
def create_feature(id,region_name,region_type):
    text=f'''
###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#region.{id}
:region.{id} rdf:type owl:NamedIndividual ,
                        :AdministrativeRegion_Province ;
               geosparql:hasGeometry :geometry.polygon.region.{id} ;
               :RegionID {id} ;
               :RegionName "{region_name}" ;
               :RegionType "{region_type}" ;
               rdfs:label "{region_name}" .
'''
    return text

# 整理市级行政区

创建Feature要素的时候，要加上Area面积属性

需要多加一个省和市的包含关系


In [1]:
# 创建要素
def create_feature(id,region_name,region_type,region_area):
    text=f'''
###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#region.{id}
:region.{id} rdf:type owl:NamedIndividual ,
                        :AdministrativeRegion_Province ;
               geosparql:hasGeometry :geometry.polygon.region.{id} ;
               :RegionID {id} ;
               :RegionName "{region_name}" ;
               :RegionType "{region_type}" ;
               :RegionArea {region_area} ;
               rdfs:label "{region_name}" .
'''
    return text


# 创建geometry
def create_geometry(region_id,wkt_text):
    text=f'''
###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#geometry.polygon.region.{region_id}
:geometry.polygon.region.{region_id} rdf:type owl:NamedIndividual ;
                                geosparql:asWKT """{wkt_text}"""^^geosparql:wktLiteral .
'''

    return  text

In [5]:
# 创建省和市之间的包含关系

def create_contains(province_id,city_id):
    text=f'''
:region.{province_id} geosparql:sfContains :region.{city_id} .
'''
    return text 

In [5]:
import ijson
from geomet import wkt

file=open("./rdf/市级行政区.ttl","a",encoding="utf-8")


with open('./矢量数据/市级行政区.geojson', 'r', encoding='utf-8') as f:
    objects = ijson.items(f, 'features.item')
    # 这个objects在这里就是相当于一个生成器，可以调用next函数取它的下一个值
    while True:
        try:
            # print(objects.__next__())

            data = objects.__next__()
            properties=data["properties"]
            geometry = data['geometry']
            towkt = wkt.dumps(geometry, decimals=8)
            # geometry=create_geometry(properties["市代码"],towkt)
            # feature=create_feature(properties["市代码"],properties["市"],properties["类型"],properties["area"])
            contains=create_contains(properties["省代码"],properties["市代码"])
            # file.write(geometry)
            # file.write(feature)
            file.write(contains)

        except StopIteration as e:
            print("数据读取完成")
            break

file.close()

数据读取完成


# 整理县级行政区

In [6]:
import ijson
from geomet import wkt

file=open("./rdf/县级行政区.ttl","a",encoding="utf-8")


with open('./矢量数据/县级行政区.geojson', 'r', encoding='utf-8') as f:
    objects = ijson.items(f, 'features.item')
    # 这个objects在这里就是相当于一个生成器，可以调用next函数取它的下一个值
    while True:
        try:
            # print(objects.__next__())

            data = objects.__next__()
            properties=data["properties"]
            geometry = data['geometry']
            towkt = wkt.dumps(geometry, decimals=8)
            geometry=create_geometry(properties["PAC"],towkt)
            feature=create_feature(properties["PAC"],properties["NAME"],properties["类型"])
            contains=create_contains(properties["市代码"],properties["PAC"])
            file.write(geometry)
            file.write(feature)
            file.write(contains)

        except StopIteration as e:
            print("数据读取完成")
            break

file.close()

数据读取完成


# 补充Geometry的类型，应该是http://www.opengis.net/ont/sf#Polygon.

In [ ]:
# 补充geometry的类型属性，
# :geometry.polygon.region.{region_id} rdf:type http://www.opengis.net/ont/sf#Polygon.


In [6]:
import ijson
from geomet import wkt

file=open("./rdf/行政区几何补充.ttl","a",encoding="utf-8")


with open('./矢量数据/省级行政区.geojson', 'r', encoding='utf-8') as f:
    objects = ijson.items(f, 'features.item')
    # 这个objects在这里就是相当于一个生成器，可以调用next函数取它的下一个值
    while True:
        try:
            data = objects.__next__()
            properties=data["properties"]
            provinceID=properties["省代码"]
            text=f''':geometry.polygon.region.{provinceID} rdf:type <http://www.opengis.net/ont/sf#Polygon>.
'''
            file.write(text)

        except StopIteration as e:
            print("省级行政区数据读取完成")
            break

with open('./矢量数据/市级行政区.geojson', 'r', encoding='utf-8') as f:
    objects = ijson.items(f, 'features.item')
    # 这个objects在这里就是相当于一个生成器，可以调用next函数取它的下一个值
    while True:
        try:
            data = objects.__next__()
            properties=data["properties"]
            cityID = properties["市代码"]
            text=f''':geometry.polygon.region.{cityID} rdf:type <http://www.opengis.net/ont/sf#Polygon>.
'''
            file.write(text)

        except StopIteration as e:
            print("市级行政区数据读取完成")
            break

with open('./矢量数据/县级行政区.geojson', 'r', encoding='utf-8') as f:
    objects = ijson.items(f, 'features.item')
    # 这个objects在这里就是相当于一个生成器，可以调用next函数取它的下一个值
    while True:
        try:
            data = objects.__next__()
            properties=data["properties"]
            countyID=properties["PAC"]
            text=f''':geometry.polygon.region.{countyID} rdf:type <http://www.opengis.net/ont/sf#Polygon>.
'''
            file.write(text)

        except StopIteration as e:
            print("县级行政区数据读取完成")
            break


file.close()

省级行政区数据读取完成
市级行政区数据读取完成
县级行政区数据读取完成
